In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
import numpy as np
import re

In [90]:
fasta_string = open('myseq.fa').read()

In [91]:
result_handle = NCBIWWW.qblast('blastn','nt',fasta_string) # 'blastn'-program to use; 'nt'-which database to search 

In [92]:
print(fasta_string)

TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTAC
AATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCAC
CTACGGTAGAG


In [93]:
blast_record = NCBIXML.read(result_handle)

In [94]:
len(blast_record.alignments)

50

In [99]:
E_VALUE_MIN = 0.01; i=0; evals=[];
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        evals.append(hsp.expect)
        if hsp.expect < E_VALUE_MIN:
            title = alignment.title;
            length = alignment.length;
            E_VALUE_MIN = hsp.expect;
print('****Alignment****')
print('sequence : ', title)
print('length : ', length)

****Alignment****
sequence :  gi|1241191387|gb|MF579563.1| Nicotiana attenuata mitochondrion, complete genome
length :  394341
e value :  2.90531e-96
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG


In [118]:
fasta_string = open('seq1.fa').read()
split = fasta_string.split('\n')
fasta_string = ''.join(split)
myseq = Seq(fasta_string)

In [119]:
len(fasta_string)

200

## Mini-test

In [2]:
records = {}
fasta_file = open('dna2.fasta')
for line in fasta_file:
    line = line.rstrip()
    if line.startswith('>'):
        words = line.split()
        name = words[0][1:]
        records[name] = ''
    else:
        records[name] = records[name] + line
fasta_file.close()
keys = list(records.keys())

**1)** How many records are in the file? - _18_

In [3]:
len(records)

18

**2)** What is the length of the longest sequence in the file? - _4894_ <br>
**3)** What is the length of the shortest sequence in the file? - _115_

In [4]:
lengths = list(map(lambda seq: len(seq), records.values()))
print(min(lengths),' ',max(lengths),' ',lengths)

115   4894   [4635, 1151, 4894, 3511, 4076, 2867, 442, 890, 967, 4338, 1352, 4564, 4804, 964, 2095, 1432, 115, 2646]


**CAUTION**: задания 4)-6) не решены. Вернусь к ним, когда разберусь.

**4)** What is the length of the longest ORF appearing in reading frame 2 of any of the sequences? - _1458_

In [42]:
def ORFs_n(sequence):
    startP = re.compile('ATG'); longest = (0,); orfs=[[0,0,0],[0,0,0]]; starts=[[0,0,0],[0,0,0]];
    for direction,nuc in [(0, Seq(sequence)), (1, Seq(sequence).reverse_complement())]:
        for frame in range(3):
            nuc = nuc[frame:]
            for m in startP.finditer(str(nuc)):
                length = 3*((len(nuc)-m.start()) // 3)
                if len(nuc[m.start():m.start()+length].translate(to_stop=True)) > orfs[direction][frame]:
                    pro = nuc[m.start():m.start()+length].translate(to_stop=True)
                    orfs[direction][frame] = len(pro); starts[direction][frame] = m.start()
    return(orfs[0]+orfs[1])

In [21]:
def ORFs_new(sequence):
    orfs=[[0,0,0],[0,0,0]]; starts=[[0,0,0],[0,0,0]];
    for direction,value in [(0, Seq(sequence)), (1, Seq(sequence).reverse_complement())]:
        for frame in range(3):
            length = 3*((len(value)-frame) // 3)
            nuc = value[frame:frame+length].translate()
            for seq in nuc.split('*'):
                m = seq.find('M')
                if ((m != -1) & ((len(seq)-m)>orfs[direction][frame])):
                    orfs[direction][frame] = len(seq)-m
                    starts[direction][frame] = nuc.find(seq)+frame-m          
    return(starts[0]+starts[1])       

In [4]:
def ORFs(sequence):
    orfs=[[0,0,0],[0,0,0]]; starts=[[0,0,0],[0,0,0]];
    for direction,value in [(0, Seq(sequence)), (1, Seq(sequence).reverse_complement())]:
        for frame in range(3):
            nuc = value[frame:]; start=0;
            while (start < len(nuc)):
                start = nuc.find('ATG', start, len(nuc)) # search for start-codon
                if (start != -1): # start-codon found
                    length = 3*((len(nuc)-start) // 3) # multiple of 3
                    translation = nuc[start:start+length].translate(table=1) # try to translate
                    stop = translation.find('*') # search for first stop-codon
                    if (stop != -1): # stop-codon found
                        length = (stop+1)*3
                        if (orfs[direction][frame] < length):
                            orfs[direction][frame] = length; starts[direction][frame] = start+1+frame
                        else:
                            length=0;
                    start = start+length+1;
                else:
                    start = len(nuc)
    return(orfs[0]+orfs[1])        

In [5]:
ORFs_for_dnas = np.array(list(map(lambda dna: ORFs(dna), records.values() )))

In [6]:
ORFs_for_dnas

array([[1296, 1296, 1296, 1632, 1632, 1632],
       [   0,    0,    0,  108,  108,  108],
       [1443, 1443, 1443, 1299, 1299, 1299],
       [2073, 2073, 2073, 2268, 2268, 2268],
       [1200, 1200, 1200, 1101, 1101, 1101],
       [1194, 1194, 1194,    0,    0,    0],
       [ 189,  189,  189,    0,    0,    0],
       [ 120,  120,  120,    0,    0,    0],
       [ 159,  159,  159,  102,  102,  102],
       [ 711,  711,  711, 1233, 1233, 1233],
       [ 204,  204,  204,   87,   87,   87],
       [1002, 1002, 1002, 1566, 1566, 1566],
       [1644, 1644, 1644, 1656, 1656, 1656],
       [ 132,  132,  132,  654,  654,  654],
       [ 210,  126,  126,  963,  963,  963],
       [ 204,  204,  204,  336,  336,  336],
       [   0,    0,    0,    0,    0,    0],
       [1344, 1344, 1344, 1014, 1014, 1014]])

**5)** What is the starting position of the longest ORF in reading frame 3 in any of the sequences? - _636_

**6)** What is the length of the longest ORF appearing in any sequence and in any forward reading frame? - _2394_

**7)** What is the length of the longest forward ORF that appears in the sequence with the identifier gi|142022655|gb|EQ086233.1|16? - _1644_

In [8]:
ID = 'gi|142022655|gb|EQ086233.1|16' # given ID, suppose
ind = keys.index(ID)
max(ORFs(records[ID])[:3])

1644

**8)** Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all? - _153_

In [36]:
import re

In [37]:
def Most_freq(n):
    repeats={}
    for value in records.values():
        for i in np.arange(0,len(value)-n+1,1):
            fragment = value[i:i+n]
            if repeats.__contains__(fragment):
                continue
            else:
                repeats[fragment] = 0
                for seq in records.values():
                    repeats[fragment] = repeats[fragment] + len(re.findall('(?='+fragment+')', seq))
    keys_rep = list(repeats.keys()); values_rep = list(repeats.values())
    return([keys_rep, values_rep])

In [38]:
[keys, values] = Most_freq(6)

In [41]:
values.sort(); print(max(values))

153


**9)** Find all repeats of length 12 in the input file. How many different 12-base sequences occur Max times? - _4_

In [42]:
[keys, values] = Most_freq(12)

In [50]:
len(np.where(np.array(values) == max(values))[0])

4

**10)** Which one of the repeats of length 7 has a maximum number of occurrences? - _CGCGCCG_

In [51]:
[keys, values] = Most_freq(7)

In [56]:
i = np.where(np.array(values) == max(values))[0][0]
print(keys[i])

CGCGCCG
